In [ ]:
# 데이턱 읽어오기 
# pandas reda_csv() 함수 이용해 파일 읽어와 CNN 모델 학습 필요한 Q(질문), label(감정) 데이터를 features, labels 리스트 저장 
train_file = './data/ChatbotData.csv'
data = pd.read_csv(train_file, delimiter=',') 
features = data['Q'].tolist() 
labels = data['label'].tolist() 

# 단어 인덱스 시퀀스 벡터 
# 위에서 불러온 질문 리스트(features) 에서 문장 하나씩 꺼내와 text_to_word_sequence() 함수 이용해 단어 시퀀스 만듦
# 단어시퀀스 = 단어 토큰들의 순차적 리스트 의미 
# 단어 시퀀스를 말뭉치(corpus) 리스트에 저장
# 텐서플로 토크나이저 texts_to_sequences() 함수 이용해 문장 내 모든 단어를 시퀀스 번호로 변환 
# 변환된 시퀀스 번호 이용해 단어 임베딩 벡터 만들
# 시퀀스 번호로 만든 벡터 문제: 문장 길이가 제각각 벡터 크기 다름 > CNN 모델 입력층 고정된 개수 입력 노드 가짐 
# 시퀀스 번호로 변환된 전체 벡터 크기를 동일하게 맞춰줘야 함
# MAX_SEQ_LEN = 15 보다 작은 벡터에는 남는 공간 생김 여길 0으로 채우는 작업 = 패딩 처리 
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features] 
tokenizer = preprocessing.text.Tokenizer() 
tokenizer.fit_on_texts(corpus) 
sequences = tokenizer.texts_to_sequences(corpus) 
word_index = tokenizer.word_index 

MAX_SEQ_LEN = 15  # 단어 시퀀스 벡터 크기 
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post') 

# 학습용, 검증용, 테스트용 데이터셋 생성
# 학습셋:검증셋:테스트셋 = 7:2:1 
# 패딩처리된 시퀀스(padded_seq) 벡터 리스트와 감정 (labels) 리스트 전체를 데이터셋 객체로 만듦 
# 데이터 랜덤 섞은 후 학습용, 검증용, 테스트용 데이터셋 7:2:1 나눠 실제 학습에 필요한 데이터셋 객체 분리 
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels)) 
ds = ds.shuffle(len(features)) 

train_size = int(len(padded_seqs) * 0.7) 
val_size = int(len(padded_seqs) * 0.2) 
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20) 
val_ds = ds.skip(train_size).take(val_size).batch(20) 
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)  

# 하이퍼파라미터 설정 
dropout_prob = 0.5 
EMB_SIZE = 128 
EPOCH = 5 
VOCAB_SIZE = len(word_index) + 1   # 전체 단어 수 

# CNN 모델 정의 
# 케라스 함수형 모델 방식으로 구현 
# 문장을 감정 클래스로 분류하는 CNN 모델은 전처리된 입력 데이터를 단어 임베팅 처리하는 영역 / 합성곱 필터와 연산을 통해 문장의 특징 정보(특징맵) 추출하고 평탄화 하는 영역 
# 완전 연결 계층 통해 감정별로 클래스 분류하는 영역

# 단어 임베딩 영역 코드 
# 입력 계층은 케라스 Input() 함수 생성 - shape 인자로 입력 노드에 들어올 데이터 형상 지정 
input_layer = Input(shape = (MAX_SEQ_LEN)) 
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer) # 임베딩 계층 
dropout_emb = Dropout(rate = dropout_prob)(embedding_layer) # 50% 확률로 Dropout() 생성 > 오버피팅 대비

# 임베딩 벡터에서 특징 추출하는 영역 구현
conv1 = Conv1D(
    filters = 128,
    kernel_size = 3, 
    padding = 'valid',
    activation = tf.nn.relu)(dropout_emb) 
pool1 = GlobalMaxPool1D()(conv1)    # 최대 풀링 연산 수행

conv2 = Conv1D(
    filters = 128,
    kernel_size = 4, 
    padding = 'valid',
    activation = tf.nn.relu)(dropout_emb) 
pool2 = GlobalMaxPool1D()(conv2) 

conv3 = Conv1D(
    filters = 128,
    kernel_size = 5, 
    padding = 'valid',
    activation = tf.nn.relu)(dropout_emb) 
pool3 = GlobalMaxPool1D()(conv3) 

# 3, 4, 5 - gram 이후 합치기 
concat = concatenate([pool1, pool2, pool3]) # 병렬로 처리된 합성곱 계층 특징맵 결과를 하나로 묶어줌 

# 완전 연결 계층 구현
hidden = Dense(128, activation=tf.nn.relu)(concat)    # 128개 출력 노드, relu 활성화 함수 사용하는 Dense 계층 생성 3개의 특정맵 데이터 입력으로 ㅂ다음
dropout_hidden = Dropout(rate = dropout_prob)(hidden) 
logits = Dense(3, name='logits')(dropout_hidden)      # 3가지 감정 분류. 출력노드 3개인 Dense() 생성. 활성화 함수 X logits = 점수
predictions = Dense(3, activation=tf.nn.softmax)(logits)  # softmax = 확률계산 

# 모델 생성 
# 위에서 정의 한 계층들 케라스 모델에 추가하는 작업 
model = Model(inputs=input_layer, outputs=predictions) 
model.compile(optimizer='adam',  # 최적화
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) 

# 모델 학습 
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1) # (학습용, 검증용, 에포크, verbose = 1 이면 모델 학습 시 진행과정 보여줌)

# 모델 평가 (테스트 데이터텟 이용) 
loss, accuracy = model.evaluate(test_ds, verbose=1) 
print('Accuracy: %f' % (accuracy * 100)) 
print('loss: %f' % (loss)) 

# 모델 저장 
model.save('cnn_model.h5') 


# evaluate() 함수 이용해 성능 평가 
# 테스트용 데이터 셋 이용 
loss, accuracy = model.evaluate(test_ds, verbose = 1) 
print('Accuracy: %f' % (accuracy * 100))
print('loss : %f' % (loss)) 

# 학습 완료 h5 파일 포맷으로 저장
model.save('cnn_model.h5') 